In [1]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import Saver
from src.plotting import analysis
from src.utils import physics

## Loading results

In [2]:
dir_path = Path('results/')
name = 'test_thruster.h5'

In [3]:
store = pd.HDFStore(dir_path/name)

In [4]:
# defining each variable
collisions_per_cell = store['collisions_per_cell'] 
df = store['df']
pmax_per_cell = store['pmax_per_cell']
averages_per_cell = store['averages_per_cell']
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']

In [5]:
# particle mass 
mass = physics.get_mass_part(53, 53, 74) # mass of the particle
# time step
dt = 1e-6 
# ratio between real particles and simulated macro-particles
mr = 32000000


verbose=False

## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing frames to plot

In [6]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.8*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Particles analysis
### Plotting number of particles evolution

In [7]:
fig, ax = plt.subplots()
analysis.nb_particles_evolution(ax, df, times = dt*unique_index);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean number of particles per cell

In [8]:
averages = averages_per_cell.groupby(averages_per_cell.index).sum() # returns a pandas Series

In [9]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(averages.index*dt, averages);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Final state

In [10]:
fig, ax = plt.subplots()
analysis.state(ax, df.loc[df.index == frames[-1]], c = None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [11]:
fig, axes = analysis.velocity_distribution(df, frames, bins = 50, density = True, sharex = False, sharey = False);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity axis distribution per *cell*

In [12]:
# spatial_hist2d plots an hist per cell defined by the params below
# it is a 2D plot of size y_res * x_res
# where each plot (i,j) takes the particle between i*x_step < x < (i+1)*x_step and j*y_step < y < (j+1)*y_step
# for i varying between between 0 and x_res-1, and j varying between between 0 and y_res-1
# You should not use too many cells because it quickly becomes very hard to view.
# Params
x_res = 3  # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001 

# Plotting - vx
fig, axes = analysis.spatial_hist2d(df, frames, val = 'vx', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Plotting - vy
fig, axes = analysis.spatial_hist2d(df, frames, val = 'vy', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## DSMC collisions analysis

### Particles collisions analysis

In [14]:
# start by summing over all cells
collisions = collisions_per_cell.groupby(collisions_per_cell.index).sum() # returns a pandas Series
print('Total number of collision :', np.sum(collisions));

Total number of collision : 19446.0


In [15]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions.index*dt, collisions);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean acceptance probability

In [16]:
proba = total_proba/collisions
print('Mean proba : {:e}'.format(np.mean(proba)))
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(proba.index*dt, proba, label = 'proba');

Mean proba : 1.799616e-01


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Maximum probability

In [17]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum() # returns a pandas Series

In [18]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(pmax.index*dt, pmax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean distance between two colliding particles

In [19]:
fig, ax = plt.subplots()
dist = total_distance/collisions
print('Mean distance : {:e} m'.format(np.mean(dist)))
analysis.set_axis(ax, x = 'time', y = 'distance')
ax.plot(dist.index*dt, dist);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean distance : 4.055559e-04 m


## Out particle analysis

In [20]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Bonus

Next is the plot of the system for each saved step. 

In [24]:
# creating speed_norm (for the color)
df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df):
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['speed_norm'])
    
    ax.set_title('{}/{}'.format(num+1, iterations), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['speed_norm'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)

ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
ax.axis('equal')

interval = 40 # 25 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=iterations, fargs=(df, ), save_count=iterations)
anim.save(dir_path/'system_state_evo.mp4', dpi = 300);